In [1]:
#!apt-get update
#!apt-get install -y tesseract-ocr
#!apt-get install -y libtesseract-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,512 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubun

In [2]:
#!pip install fitz
#!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


In [3]:
#pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 48.2 MB/s eta 0:00:00


In [4]:
#pip install pytesseract

In [5]:
#!pip install spacy
#!pip install transformers
#!pip install pdf2image
#!pip install Pillow


## **DOCUMENT CONVERSION AND OCR INTEGRATION**

In [6]:
import fitz
import pytesseract
import cv2
import numpy as np

def convert_pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def extract_images_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        for img in page.get_images(full=True):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            images.append(image_bytes)
    return images


def ocr_image(image_bytes, languages=None):
    if languages is None:
        languages = ['eng']
    elif isinstance(languages, str):
        languages = [languages]

    image_np = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
    image = Image.fromarray(image_np)
    text = pytesseract.image_to_string(image, lang='+'.join(languages))
    return text



In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **PREPROCESSING & TOKENIZATION**

In [8]:
import re
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

def segment_sentences(text):
    return sent_tokenize(text)

def tokenize_sentence(sentence):
    return word_tokenize(sentence)

def preprocess_text(text):
    text = clean_text(text)
    sentences = segment_sentences(text)
    tokenized_sentences = [tokenize_sentence(sent) for sent in sentences]

    return tokenized_sentences


In [9]:
import os
from huggingface_hub import hf_hub_download

In [10]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")

In [11]:
#pip install Translate

## **LLM INTEGRATION**

In [12]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from translate import Translator
from PIL import Image

In [32]:
def load_llm_model(model_name):
    return pipeline("text-classification", model=model_name)

def classify_document(text):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs

def translate_text(text, target_language="de"):
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")
    translated_chunks = []
    chunk_size = 512
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        translated_chunk = translator(chunk)[0]['translation_text']
        translated_chunks.append(translated_chunk)
    return ' '.join(translated_chunks)



def extract_information(text):
    ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
    entities = ner_model(text)

    doc = nlp(text)
    spacy_entities = [(ent.text, ent.label_) for ent in doc.ents]

    return entities, spacy_entities


def process_pdf(uploaded_file):
    pdf_path = uploaded_file.name
    with open(pdf_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    text = convert_pdf_to_text(pdf_path)
    images = extract_images_from_pdf(pdf_path)
    for img_bytes in images:
        img_text = ocr_image(img_bytes, languages=['eng', 'spa', 'deu'])
        text += "\n" + img_text
    return text

## **USERFRIENDLY CHATBOT USING STREAMLIT**

In [26]:
import streamlit as st

st.title("Document Processing Chatbot")

uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
if uploaded_file is not None:
    with st.spinner("Processing..."):
        text = process_pdf(uploaded_file)
        preprocessed_text = preprocess_text(text)
        st.success("File processed successfully!")
        st.text_area("Extracted Text", text, height=300)

    if st.button("Extract Information"):
        with st.spinner("Extracting information..."):
            entities, spacy_entities = extract_information(text)
            st.write("Entities (NER):", entities)
            st.write("Entities (spaCy):", spacy_entities)

    if st.button("Classify Document"):
        with st.spinner("Classifying document..."):
            classification = classify_document(text)
            st.json(classification)

    if st.button("Translate Text"):
        with st.spinner("Translating text..."):
            translation = translate_text(text)
            st.text_area("Translated Text", translation, height=300)


2024-06-07 11:46:14.766 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [27]:
#!pip install streamlit -q

In [28]:
!wget -q -O - ipv4.icanhazip.com

34.68.96.143


In [34]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.68.96.143:8501

npx: installed 22 in 1.956s
  Stopping...
^C
